In [2]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os
import time
import re
print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [1]:
def compute_D1_psnr(original_mesh, decoded_mesh):
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)

    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)

    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)

    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("D1 mse:",MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    #print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    #print(psnr)
    return psnr


def compute_D2_psnr(original_mesh, decoded_mesh):
    decoded_mesh.compute_vertex_normals()

    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)

    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)

    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_decoded.normals = o3d.utility.Vector3dVector(decoded_mesh.vertex_normals)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)

    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(
            np.dot((original_vertices[i] - decoded_vertices[index])[0], np.array(pcd_decoded.normals)[index][0]))
    MSE = MSE / len(original_vertices)

    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)

    return psnr


def compute_MSE_RMSE(original_mesh, decoded_mesh):
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)

    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)

    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)

    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("MSE:", MSE)
    RMSE = np.sqrt(MSE)

    return np.log10(MSE), np.log10(RMSE)


from scipy.spatial.distance import directed_hausdorff


def compute_Hausdorff(original_mesh, decoded_mesh):
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    hausdorff = directed_hausdorff(original_vertices, decoded_vertices)
    return hausdorff[0] * 1e4


In [22]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [30]:
dataset = "answering"
for i in range(1, 10):
    input_static_mesh_path = fr'G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0{i:03}_raw.obj'
    output_path = fr'G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0{i:03}_raw.drc'
    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_encoder',
        '-i', input_static_mesh_path,
        '-o', output_path,
        '-qp', str('14'),
        '-cl', '7'
    ], capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_decoder',
        '-i', output_path,
        '-o',
        fr'G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0{i:03}_raw.drc.obj'
    ], capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0001_raw.drc (820 ms to encode).

Encoded size = 222541 bytes

For better compression, increase the compression level up to '-cl 10' .



Decoded geometry saved to G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0001_raw.drc.obj (166 ms to decode)


Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0002_raw.drc (754 ms to encode).

Encoded size = 223930 bytes

For better compression, increase the compression level up to '-cl 10' .



Decoded geometry saved to G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0002_raw.drc.obj (167 ms to decode)


Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\ChromeDownloads\Raw\Mesh\static/evaluation/mesh_0003_raw.drc (787 ms to encode).

Encoded

In [24]:
input_reference_mesh_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset2}_2000\reference_mesh/decimated_reference_mesh.obj'
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(input_reference_mesh_path, enable_post_processing=False)
print(input_decimated_reference_mesh)
np.array(input_decimated_reference_mesh.vertices)

TriangleMesh with 5819 points and 12000 triangles.


array([[-0.692759, -0.930068, -1.81081 ],
       [-0.687829, -0.87446 , -1.8103  ],
       [-0.676114, -0.760236, -1.76497 ],
       ...,
       [-0.701636, -0.279573, -0.486458],
       [-0.200061, -0.169802,  0.588358],
       [-0.643394, -0.397644,  0.274097]])

In [25]:
dataset = "answering"
input_reference_mesh_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}_2000\reference_mesh/decimated_reference_mesh.obj'

output_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}_2000\reference_mesh/encoded_decimated_reference_mesh.drc'
result = subprocess.run([
    r'G:\Github\draco\build\Debug\draco_encoder',
    '-i', input_reference_mesh_path,
    '-o', output_path,
    '-qp', str('14'),
    '-cl', '7'
], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)
result = subprocess.run([
    r'G:\Github\draco\build\Debug\draco_decoder',
    '-i', output_path,
    '-o',
    fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}_2000\reference_mesh/decode_decimated_reference_mesh.obj'
], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/encoded_decimated_reference_mesh.drc (60 ms to encode).

Encoded size = 24355 bytes

For better compression, increase the compression level up to '-cl 10' .



Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/decode_decimated_reference_mesh.obj (16 ms to decode)




In [26]:
dataset = "answering_2000"
dataset2 = "answering"
GoF = 10
qp = 8
times = []
for i in range(0, 9):
    offset = 1
    print(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset2}_{i + offset:03}.ply')
    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_encoder',
        #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
        '-point_cloud',
        '-i',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset2}_{i + offset:03}.ply',
        '-o',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset2}_{i + offset:03}.drc',
        '-qp', str(qp),
        '-cl', '10'
    ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to encode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))

    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_encoder',
        #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
        '-point_cloud',
        '-i',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset2}_{i + offset:03}.ply',
        '-o',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset2}_{i + offset:03}_0.drc',
        '-qp', str(qp),
        '-cl', '0'
    ], capture_output=True, text=True)
    print(result.stdout)

if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")
times = []
for i in range(0, GoF):
    offset = 1
    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_decoder',
        '-i',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset2}_{i + offset:03}_0.drc',
        '-o',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset2}_{i + offset:03}_displacements.ply'
    ], capture_output=True, text=True)
    print(result.stdout)

    result = subprocess.run([
        r'G:\Github\draco\build\Debug\draco_decoder',
        '-i',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset2}_{i + offset:03}.drc',
        '-o',
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset2}_{i + offset:03}_displacements_10.ply'
    ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to decode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))

if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")

G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/dis_answering_001.ply
Encoder options:
  Compression level = 10
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/GoF10/dis_answering_001.drc (23 ms to encode).

Encoded size = 8293 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/GoF10/dis_answering_001_0.drc (6 ms to encode).

Encoded size = 28684 bytes

For better compression, increase the compression level up to '-cl 10' .


G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\answering_2000\reference_mesh/dis_answering_002.ply
Encoder options:
  Compression level = 10
  Positions: Quantization = 8 bits

Encoded point cloud saved to G:\VS2022Proj

In [29]:
def calculate_bitrate(file_size, duration):
    return file_size * 8 / duration

dataset = "answering_2000"
dataset2 = "answering"
GoF = 10
number_frames = 9
frame_rate = 30
total_size = 0
offset = 1
for i in range(0, 9):
    displacement_file_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/dis_{dataset2}_{i + offset:03}_0.drc'
    displacement_file_size = os.path.getsize(displacement_file_path)
    total_size += displacement_file_size
reference_mesh_file_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/encoded_decimated_reference_mesh.drc'
reference_mesh_file_size = os.path.getsize(reference_mesh_file_path)
total_size += reference_mesh_file_size

static_mesh_file_path = fr'G:\ChromeDownloads\Raw\Mesh\static/mesh_0000_raw.drc'
static_mesh_file_size = os.path.getsize(static_mesh_file_path)
total_size += static_mesh_file_size


total_duration = number_frames / frame_rate
overall_bitrate = calculate_bitrate(total_size, total_duration)

print(f"Total Size of {number_frames} DRC Files: {total_size} bytes")
print(f"Overall Bitrate: {overall_bitrate} bits per second")

bitrate_kbps = overall_bitrate / 1000
bitrate_mbps = overall_bitrate / 1000000

print(f"Overall Bitrate: {bitrate_kbps:.2f} Kbps")
print(f"Overall Bitrate: {bitrate_mbps:.2f} Mbps")

Total Size of 9 DRC Files: 925676 bytes
Overall Bitrate: 24684693.333333336 bits per second
Overall Bitrate: 24684.69 Kbps
Overall Bitrate: 24.68 Mbps


In [28]:
original_displacements = []
decoded_displacements = []
dis_plys = []
for i in range(0, 9):
    original_displacement = np.loadtxt(
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\Answering\reference/displacements_{dataset2}_{i + offset:03}.txt')
    decoded_displacement = o3d.io.read_point_cloud(
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset2}_{i + offset:03}_displacements.ply')
    dis_ply = o3d.io.read_point_cloud(
        fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{GoF}/decoded_{dataset2}_{i + offset:03}_displacements.ply')
    original_displacements.append(original_displacement)
    decoded_displacements.append(decoded_displacement)
    dis_plys.append(dis_ply)
    
d1s = []
d2s = []
mses = []
rmses = []
hausdorffs = []
offset = 1
original_static = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\static/mesh_0000_raw.drc.obj', enable_post_processing=False)
for m in range(0, 9):
    decode_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
    #print(decode_decimated_reference_mesh)
    np.array(decode_decimated_reference_mesh.vertices)
    start = time.time()
    subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
    mesh = deepcopy(subdivided_decoded_mesh)
    triangles = deepcopy(mesh.triangles)
    end = time.time()
    print("subdivision time:", end - start)
    #print(subdivided_decoded_mesh)
    input_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
    #print(subdivided_mesh)
    #original_static = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\meshes/mitch_fr0{m+offset:03}.obj')
    #original_dancer = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\meshes/dynamic_mesh_0{m+offset:03}.obj')
    original_dancer = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\{dataset2}/mesh_0{m+offset:03}.obj')
    #original_static = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0{m+offset:03}_decoded.obj', enable_post_processing=False)
    #print(original_dancer)
    decoded_mesh_vertices = np.array(decode_decimated_reference_mesh.vertices)
    subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)
    
    
    displacement = np.array(decoded_displacements[m].points)
    
    dis_indexer = o3d.geometry.PointCloud()
    dis_indexer.points = o3d.utility.Vector3dVector(original_displacements[m])
    dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)
    
    pcd_indexer = o3d.geometry.PointCloud()
    pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)
    
    reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)
    start = time.time()
    for i in range(0, len(subdivided_decoded_mesh_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
        [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[m][index[0]], 1)
        #print(displacement[dis_index], original_displacements[index[0]])
        reordered_vertices[i] += displacement[dis_index[0]]
    end = time.time()
    print("rematching time:", end - start)
    reconstruct_mesh = o3d.geometry.TriangleMesh()
    reconstruct_mesh.triangles = subdivided_decoded_mesh.triangles
    reconstruct_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)
    reconstruct_mesh += original_static
    reconstruct_mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([reconstruct_mesh])
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\scene_compression\Results\decode_Ours\{dataset2}/GoF{10}/decoded_{dataset2}_fr0{m+offset:03}.obj', reconstruct_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    d1 = max(compute_D1_psnr(original_dancer, reconstruct_mesh), compute_D1_psnr(reconstruct_mesh, original_dancer))
    print("D1:", d1)
    d1s.append(d1)
    
    #d2 = max(compute_D2_psnr(original_dancer, reconstruct_mesh), compute_D2_psnr(reconstruct_mesh, original_dancer))
    #print("D2:", d2)
    #d2s.append(d2)

    #logmse1, logrmse1 = compute_MSE_RMSE(original_dancer, reconstruct_mesh)
    #logmse2, logrmse2 = compute_MSE_RMSE(reconstruct_mesh, original_dancer)
    #logmse = min(logmse1, logmse2)
    #logrmse = min(logrmse1, logrmse2)
    #print("log10 of mse:", logmse, ", log10 of rmse:", logrmse)
    #mses.append(logmse)
    #rmses.append(logrmse)
    
    #hausdorff = compute_Hausdorff(original_dancer, reconstruct_mesh)
    #print("Hausdorff distance:", hausdorff)
    #hausdorffs.append(hausdorff)
o3d.visualization.draw_geometries([reconstruct_mesh])
print("average D1:", np.mean(d1s))
#print("average D2:", np.mean(d2s))
print("average log10 of mse:", np.mean(mses))
print("average log10 of rmse:", np.mean(rmses))
print("average Hausdorff:", np.mean(hausdorffs))

subdivision time: 0.0020084381103515625
rematching time: 0.15724968910217285
[Open3D WARNING] Write OBJ can not include triangle normals.


KeyboardInterrupt: 